In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import itertools

In [3]:
# Verificar si CUDA está disponible
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(9, 60)  
        self.fc2 = nn.Linear(60, 30)  
        self.fc3 = nn.Linear(30, 20)
        self.fc4 = nn.Linear(20, 1)    
        self.sigmoid = nn.Sigmoid()  # Función de activación sigmoide

    def forward(self, x):
        x = self.sigmoid(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        x = self.sigmoid(self.fc4(x))
        return x

In [5]:
model = Net()
model.to(device) # Mover el modelo a la GPU

Net(
  (fc1): Linear(in_features=9, out_features=60, bias=True)
  (fc2): Linear(in_features=60, out_features=30, bias=True)
  (fc3): Linear(in_features=30, out_features=20, bias=True)
  (fc4): Linear(in_features=20, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [6]:
enteros = range(-4, 5)
binarios = [list(map(int, list(bin(i)[2:].zfill(8)))) for i in range(256)]

combinaciones = list(itertools.product(binarios, enteros))

X = np.array([list(comb[0]) + [comb[1]] for comb in combinaciones])
y = np.array([(comb[1] == 0) or (comb[0].count(0) == comb[0].count(1)) for comb in combinaciones], dtype=int)

# Imprimir conjunto de datos X y etiquetas y
for i in range(len(X)):
    print(f'X: {X[i]}, y: {y[i]}')

X = torch.tensor(X, dtype=torch.float32).to(device)
y = torch.tensor(y, dtype=torch.float32).to(device)

X: [ 0  0  0  0  0  0  0  0 -4], y: 0
X: [ 0  0  0  0  0  0  0  0 -3], y: 0
X: [ 0  0  0  0  0  0  0  0 -2], y: 0
X: [ 0  0  0  0  0  0  0  0 -1], y: 0
X: [0 0 0 0 0 0 0 0 0], y: 1
X: [0 0 0 0 0 0 0 0 1], y: 0
X: [0 0 0 0 0 0 0 0 2], y: 0
X: [0 0 0 0 0 0 0 0 3], y: 0
X: [0 0 0 0 0 0 0 0 4], y: 0
X: [ 0  0  0  0  0  0  0  1 -4], y: 0
X: [ 0  0  0  0  0  0  0  1 -3], y: 0
X: [ 0  0  0  0  0  0  0  1 -2], y: 0
X: [ 0  0  0  0  0  0  0  1 -1], y: 0
X: [0 0 0 0 0 0 0 1 0], y: 1
X: [0 0 0 0 0 0 0 1 1], y: 0
X: [0 0 0 0 0 0 0 1 2], y: 0
X: [0 0 0 0 0 0 0 1 3], y: 0
X: [0 0 0 0 0 0 0 1 4], y: 0
X: [ 0  0  0  0  0  0  1  0 -4], y: 0
X: [ 0  0  0  0  0  0  1  0 -3], y: 0
X: [ 0  0  0  0  0  0  1  0 -2], y: 0
X: [ 0  0  0  0  0  0  1  0 -1], y: 0
X: [0 0 0 0 0 0 1 0 0], y: 1
X: [0 0 0 0 0 0 1 0 1], y: 0
X: [0 0 0 0 0 0 1 0 2], y: 0
X: [0 0 0 0 0 0 1 0 3], y: 0
X: [0 0 0 0 0 0 1 0 4], y: 0
X: [ 0  0  0  0  0  0  1  1 -4], y: 0
X: [ 0  0  0  0  0  0  1  1 -3], y: 0
X: [ 0  0  0  0  0  0  1  1 -2], 

In [7]:
# Crear función de pérdida y optimizador
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [16]:
# Entrenar el modelo
epochs = 10000
for epoch in range(epochs):
    # Pasar datos de entrada a través del modelo y obtener predicciones
    outputs = model(X)
    
    # Calcular la pérdida
    loss = criterion(outputs, y.unsqueeze(1))
    
    # Reiniciar gradientes y realizar retropropagación
    optimizer.zero_grad()
    loss.backward()
    
    # Actualizar los pesos del modelo
    optimizer.step()
    
    # Imprimir la pérdida en cada epoch
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

Epoch [10/10000], Loss: 0.0002
Epoch [20/10000], Loss: 0.0002
Epoch [30/10000], Loss: 0.0002
Epoch [40/10000], Loss: 0.0002
Epoch [50/10000], Loss: 0.0002
Epoch [60/10000], Loss: 0.0002
Epoch [70/10000], Loss: 0.0002
Epoch [80/10000], Loss: 0.0002
Epoch [90/10000], Loss: 0.0002
Epoch [100/10000], Loss: 0.0002
Epoch [110/10000], Loss: 0.0002
Epoch [120/10000], Loss: 0.0002
Epoch [130/10000], Loss: 0.0002
Epoch [140/10000], Loss: 0.0002
Epoch [150/10000], Loss: 0.0002
Epoch [160/10000], Loss: 0.0002
Epoch [170/10000], Loss: 0.0002
Epoch [180/10000], Loss: 0.0002
Epoch [190/10000], Loss: 0.0002
Epoch [200/10000], Loss: 0.0002
Epoch [210/10000], Loss: 0.0002
Epoch [220/10000], Loss: 0.0002
Epoch [230/10000], Loss: 0.0002
Epoch [240/10000], Loss: 0.0002
Epoch [250/10000], Loss: 0.0002
Epoch [260/10000], Loss: 0.0002
Epoch [270/10000], Loss: 0.0002
Epoch [280/10000], Loss: 0.0002
Epoch [290/10000], Loss: 0.0002
Epoch [300/10000], Loss: 0.0002
Epoch [310/10000], Loss: 0.0002
Epoch [320/10000]

In [37]:
# Evaluar el modelo
with torch.no_grad():
    # Pasar datos de entrada a través del modelo y obtener predicciones
    outputs = torch.round(model(X))
    outputs = outputs.cpu()
    outputs = np.array(outputs)
    y = np.array(y.cpu())
    count = 0
    for i in range(outputs.shape[0]):   #chanchada
        if y[i] == outputs[i]:
            count = count + 1
    precision = count/outputs.shape[0] * 100
    print(precision)

100.0


In [41]:
for i in range(outputs.shape[0]):   #chanchada suprema
        if y[i] == outputs[i]:
            count = count + 1
        print(f'salida: {outputs[i]}, y: {y[i]}')

salida: [0.], y: 0.0
salida: [0.], y: 0.0
salida: [0.], y: 0.0
salida: [0.], y: 0.0
salida: [1.], y: 1.0
salida: [0.], y: 0.0
salida: [0.], y: 0.0
salida: [0.], y: 0.0
salida: [0.], y: 0.0
salida: [0.], y: 0.0
salida: [0.], y: 0.0
salida: [0.], y: 0.0
salida: [0.], y: 0.0
salida: [1.], y: 1.0
salida: [0.], y: 0.0
salida: [0.], y: 0.0
salida: [0.], y: 0.0
salida: [0.], y: 0.0
salida: [0.], y: 0.0
salida: [0.], y: 0.0
salida: [0.], y: 0.0
salida: [0.], y: 0.0
salida: [1.], y: 1.0
salida: [0.], y: 0.0
salida: [0.], y: 0.0
salida: [0.], y: 0.0
salida: [0.], y: 0.0
salida: [0.], y: 0.0
salida: [0.], y: 0.0
salida: [0.], y: 0.0
salida: [0.], y: 0.0
salida: [1.], y: 1.0
salida: [0.], y: 0.0
salida: [0.], y: 0.0
salida: [0.], y: 0.0
salida: [0.], y: 0.0
salida: [0.], y: 0.0
salida: [0.], y: 0.0
salida: [0.], y: 0.0
salida: [0.], y: 0.0
salida: [1.], y: 1.0
salida: [0.], y: 0.0
salida: [0.], y: 0.0
salida: [0.], y: 0.0
salida: [0.], y: 0.0
salida: [0.], y: 0.0
salida: [0.], y: 0.0
salida: [0.],

In [38]:
# Guardar los pesos del modelo en un archivo .pth
torch.save(model.state_dict(), 'indicatrizE.pth')
print("Los pesos del modelo se han guardado")

Los pesos del modelo se han guardado
